# 第12章 TensorFlow计算加速
在前面的章节中介绍了使用TensorFlow实现各种深度学习的算法。**然而要将深度学习应用到实际问题中，一个非常大的问题在于训练深度学习模型需要的计算量太大**。比如要将第6章中介绍的Inception-v3模型在单机上训练到78%的正确率需要将近半年的时间（[来源](https://research.googleblog.com/2016/04/announcing-tensorflow-08-now-with.html)，2016.04），这样的训练速度是完全无法应用到实际生产中的。为了加速训练过程，本章将介绍如何通过TensorFlow利用GPU或／和分布式计算进行模型训练。

- 首先，在12.1节中将介绍如何在TensorFlow中使用**单个GPU进行计算加速**，也将介绍生成TensorFlow会话（tf.Session）时的一些常用参数。通过这些参数可以使调试更加方便而且程序的可扩展性更好。
- 然而，在很多情况下，单个GPU的加速效率无法满足训练大型深度学习模型的计算量需求，这时将需要利用更多的计算资源。为了同时利用多个GPU或者多台机器，12.2节中将介绍**训练深度学习模型的井行方式**。
- 然后，12.3节将介绍如何**在一台机器的多个GPU上并行化地训练深度学习模型**。在这一节中也将给出具体的TensorFlow样例程序来使用多GPU训练模型，并比较并行化效率提升的比率。
- 最后在12.4节中将介绍**分布式TensorFlow**，以及如何通过分布式TensorFlow训练深度学习模型。在这一节中将给出具体的TensorFlow样例程序来实现不同的分布式深度学习训练模式。

## 12.1 TensorFlow使用GPU
**TensorFlow程序可以通过`tf.device`函数来指定运行每一个操作的设备，这个设备可以是本地的CPU或者GPU，也可以是某一台远程的服务器。**但在本节中只关心本地的设备。TensorFlow会给每一个可用的设备一个名称，`tf.device`函数可以通过设备的名称来指定执行运算的设备。比如CPU在TensorFlow中的名称为/cpu:0。在默认情况下：
- 即使机器有多个CPU, TensorFlow也不会区分它们，所有的CPU都使用/cpu:0作为名称。
- 而一台机器上不同GPU的名称是不同的，第n个GPU在TensorFlow中的名称为/gpu:n，比如第一个GPU的名称为/gpu:0，第二个为/gpu:1，以此类推。

**1. 查看运算的设备**

TensorFlow提供了一个快捷的方式来查看运行每一个运算的设备。在生成会话时，可以通过设置log_device_placement参数来打印运行每一个运算的设备。以下程序展示了如何使用log_device_placement这个参数：

In [1]:
import tensorflow as tf

a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
c = a + b

# 通过log_device_placement参数来记录运行每一个运算的设备。
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[2. 4. 6.]


*注意上述代码必须在命令行运行(文件见本文件同目录下a.py)，才会看到如下图的具体运行设备详细内容，参考[here](https://stackoverflow.com/questions/39677168/tensorflow-documentations-example-code-on-logging-device-placement-doesnt-pr)，下同。*
<p align='center'>
    <img src=images/output1.JPG>
    <center>图12-1 output1</center>
</p>

在以上代码中，TensorFlow程序生成会话时加入了参数log_device placement=True，所以程序会将运行每一个操作的设备输出到屏幕。于是除了可以看到最后的计算结果，还可以看到类似“add: (Add)/job:localhost/replica:0/task:0/device:CPU:0”这样的输出。这些输出显示了执行每一个运算的设备。比如加法操作add是通过CPU来运行的，因为它的设备名称中包含了CPU:0。

**在配置好GPU环境的TensorFlow中，如果操作没有明确地指定运行设备，那么TensorFlow会优先选择GPU**。比如将以上代码在亚马逊（Amazon Web Services, AWS）的g2.8xlarge实例上运行时，会得到类似以下的运行结果:

In [ ]:
'''
Device mapping:
/job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GRID K520, pci bus
id: 0000:00:03.0
/job:localhost/replica:0/task:0/gpu:1 -> device: 1, name: GRID K520, pci bus
id: 0000:00:04.0
/job:localhost/replica:0/task:0/gpu:2 -> device: 2, name: GRID K520, pci bus
id :0000:00:05.0
/job:localhost/replica:0/task:0/gpu:3 -> device: 3, name: GRID K520, pci bus
id :0000:00:06.0

add : (Add): /job:localhost/replica:0/task:0/gpu:0
b : (Const): /job:localhost/replica:0/task:0/gpu:0
a : (Const): /job:localhost/replica:0/task:0/gpu:0
[ 2 . 4 . 6.]
'''

以上输出中，尽管g2.8xlarge实例有4个GPU，TensorFlow只会默认将运算优先放到/gpu:0上。如上所示。

**2. 指定运算的设备**

**如果需要将某些运算放到不同的GPU或者CPU上，就需要通过`tf.device`来手工指定**。以下程序给出了一个手工指定运行设备的样例：

In [2]:
import tensorflow as tf

with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
    b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
with tf.device('/gpu:1'):
    c = a + b
    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(c))

[2. 4. 6.]


以上代码在在AWS g2.8xlarge实例上运行上述代码可以得到以下结果：

In [ ]:
'''
Device mapping:
/job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GRID K520, pci bus
id: 0000:00:03.0
/job:localhost/replica:0/task:0/gpu:1 -> device: 1, name: GRID K520, pci bus
id: 0000:00:04.0
/job:localhost/replica:0/task:0/gpu:2 -> device: 2, name: GRID K520, pci bus
id :0000:00:05.0
/job:localhost/replica:0/task:0/gpu:3 -> device: 3, name: GRID K520, pci bus
id :0000:00:06.0

add : (Add): /job:localhost/replica:0/task:0/gpu:1
b : (Const): /job:localhost/replica:0/task:0/cpu:0
a : (Const): /job:localhost/replica:0/task:0/cpu:0
[ 2 . 4 . 6.]
'''

在本人笔记本上运行（b.py），得到结果如下（由于只有一个gpu，指定为:/gpu:0，其余相同）：
<p align='center'>
    <img src=images/output2.JPG>
    <center>图12-2 output2</center>
</p>

在以上代码中可以看到生成常量a和b的操作被加载到了CPU上，而加法操作被放到了第二个GPU “/gpu:1”上。

**3. GPU释放到CPU**

**但是在TensorFlow中，不是所有的操作都可以被放在GPU上，如果强行将无法放在GPU上的操作指定到GPU上，那么程序将会报错。**以下代码给出了一个报错的样例：

In [1]:
import tensorflow as tf

# 在CPU上运行tf.Variable
a_cpu = tf.Variable(0, name="a_cpu")

with tf.device('/gpu:0'):
    # 将tf.Variable强制放在GPU上
    a_gpu = tf.Variable(0, name="a_gpu")
    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(tf.global_variables_initializer()))

InvalidArgumentError: Cannot assign a device for operation 'a_gpu': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
VariableV2: CPU 
Assign: CPU 
Identity: GPU CPU 

Colocation members and user-requested devices:
  a_gpu (VariableV2) /device:GPU:0
  a_gpu/Assign (Assign) /device:GPU:0
  a_gpu/read (Identity) /device:GPU:0

Registered kernels:
  device='CPU'
  device='GPU'; dtype in [DT_HALF]
  device='GPU'; dtype in [DT_FLOAT]
  device='GPU'; dtype in [DT_DOUBLE]
  device='GPU'; dtype in [DT_INT64]

	 [[{{node a_gpu}} = VariableV2[container="", dtype=DT_INT32, shape=[], shared_name="", _device="/device:GPU:0"]()]]

Caused by op 'a_gpu', defined at:
  File "d:\python3\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\python3\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\pyt

不同版本的TensorFlow对GPU的支持不一样，如果程序中全部使用强制指定设备的方式会降低程序的可移植性。在TensorFlow的[kernel](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/core/kernels)中定义了哪些操作可以跑在GPU上。比如可以在[variable_ops.cc](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/kernels/resource_variable_ops.cc#L469)程序中找到以下定义:

In [ ]:
'''
#define REGISTER_GPU_KERNELS(type)                                       \
  REGISTER_KERNEL_BUILDER(Name("AssignAddVariableOp")                    \
                              .Device(DEVICE_GPU)                        \
                              .HostMemory("resource")                    \
                              .TypeConstraint<type>("dtype"),            \
                          AssignUpdateVariableOp<GPUDevice, type, ADD>); \
  REGISTER_KERNEL_BUILDER(Name("AssignSubVariableOp")                    \
                              .Device(DEVICE_GPU)                        \
                              .HostMemory("resource")                    \
                              .TypeConstraint<type>("dtype"),            \
                          AssignUpdateVariableOp<GPUDevice, type, SUB>);

TF_CALL_GPU_NUMBER_TYPES(REGISTER_GPU_KERNELS);
'''

*注意：书上原本的代码，tensorflow已经更新为如上个cell所示，这里或许有些变动。*

**在这段定义中可以看到GPU只在部分数据类型上支持tf.Variable操作**。如果在TensorFlow代码库中搜索调用这段代码的宏TF_CALL_GPU_NUMBER_TYPES，可以发现在GPU上，tf.Variable操作只支持实数型（floatl6、float32和double）的参数。而在报错的样例代码中给定的参数是整数型的，所以不支持在GPU上运行。**为避免这个问题，TensorFlow在生成会话时可以指定[allow_soft_placement](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/protobuf/config.proto#L357)参数，当设置为True时，如果运算无法由GPU执行，那么TensorFlow会自动将它放到CPU上执行**。以下代码给出了一个使用allow_soft_placement参数的样例：

In [1]:
import tensorflow as tf

# 在CPU上运行tf.Variable
a_cpu = tf.Variable(0, name="a_cpu")

with tf.device('/gpu:0'):
    # 将tf.Variable强制放在GPU上
    a_gpu = tf.Variable(0, name="a_gpu")
    
# 通过allow_soft_placement参数自动将无法放在GPU上的操作放回CPU
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
sess.run(tf.global_variables_initializer())

同样在AWS g2.8xlarge上可以得到如下：

In [ ]:
'''
Device mapping:
/job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GRID K520, pci bus
id: 0000:00:03.0
/job:localhost/replica:0/task:0/gpu:1 -> device: 1, name: GRID K520, pci bus
id: 0000:00:04.0
/job:localhost/replica:0/task:0/gpu:2 -> device: 2, name: GRID K520, pci bus
id :0000:00:05.0
/job:localhost/replica:0/task:0/gpu:3 -> device: 3, name: GRID K520, pci bus
id :0000:00:06.0
a_gpu: /job:localhost/replica:0/task:0/cpu:0
a_gpu/read: /job:localhost/replica:O/task:0/cpu:0
a_gpu/Assign: /job:localhost/replica:O/task:0/cpu:0
init/NoOp_1: /job:localhost/replica:0/task:0/gpu:0
a_cpu: /job:localhost/replica:0/task:0/cpu:0
a_cpu/read: /job:localhost/replica:0/task:0/cpu:0
a_cpu/Assign: /job:localhost/replica:0/task:0/cpu:0
init/NoOp: /job:localhost/replica:0/task:0/gpu:0
init: /job :localhost/replica:0/task:0/gpu:0
a_gpu/initial_value: /job:localhost/replica:0/task:0/gpu:0
a_cpu/initial_value: /job:localhost/replica:0/task:0/cpu:0
'''

从输出的日志中可以看到在生成变量a_gpu时，无法放到GPU上的运算被自动调整到了CPU上（比如a_gpu和a_gpu/read），而可以被GPU执行的命令（比a_gpu/initial_value）依旧由GPU执行。在本人电脑下运行（c.py），输出如下：
<p align='center'>
    <img src=images/output3.JPG>
    <center>图12-3 output3</center>
</p>

**4. 使用部分GPU**

虽然GPU可以加速TensorFlow的计算，但一般来说不会把所有的操作全部放在GPU上。**一个比较好的实践是将计算密集型的运算放在GPU上，而把其他操作放到CPU上。GPU是机器中相对独立的资源，将计算放入或者转出GPU都需要额外的时间。而且GPU需要将计算时用到的数据从内存复制到GPU设备上，这也需要额外的时间。TensorFlow可以自动完成这些操作而不需要用户特别处理，但为了提高程序运行的速度，用户也需要尽量将相关的运算放在同一个设备上。**

**TensorFlow默认会占用设备上的所有GPU以及每个GPU的所有显存。**

**使用部分个数的GPU：**如果在一个TensorFlow程序中只需要使用部分GPU，可以通过设置CUDA_VISIBLE_DEVICES环境变量来控制。有三种设置方式：
- 在运行时设置这个环境变量：

In [ ]:
'''
＃只使用第二块GPU(GPU编号从0开始）。在demo_code.py中，机器上的第二块GPU的
＃名称变成/gpu:0，不过在运行时所有/gpu:0的运算将被放在第二块GPU上。
CUDA_VISIBLE_DEVICES=1 python demo_code.py

＃只使用第一块和第二块GPU
CUDA_VISIBLE_DEVICES=0,1 python demo_code.py
'''

- 在程序中设置环境变量：

In [ ]:
'''
import os

＃只使用第三块GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
'''

- GPUOptions分配，[visible_device_list](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/protobuf/config.proto#L55):

In [ ]:
'''
config = tf.ConfigProto()
config.visible_device_list = '0'
session = tf.Session(config=config, ... )

# 或：
gpu_options = tf.GPUOptions(visible_device_list='0')
sess =  tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
'''

**使用GPU的部分显存：**TensorFlow也支持动态分配GPU的显存，使得一块GPU上可以同时运行多个任务。下面给出了TensorFlow动态分配显存的两种方法：
- 按需分配，[allow_growth](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/protobuf/config.proto#L36)：

In [ ]:
'''
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config, ... )

# 或：
gpu_options = tf.GPUOptions(allow_growth=True)
sess =  tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
'''

- 按固定比例分配，[per_process_gpu_memory_fraction](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/protobuf/config.proto#L23)：

In [ ]:
'''
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4        # 直接分配40%显存
session = tf.Session(config=config, ... )

# 或：
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)   
sess =  tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
'''

更多关于GPUOptions可以参考[tensorflow源代码](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/protobuf/config.proto#L16)。